### NSMC 로드

In [1]:
from Korpora import Korpora
nsmc = Korpora.load("nsmc", force_download=True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:01, 8.54MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 7.64MB/s]                            


In [3]:
import os

# nsmc 전처리

def write_lines(path, lines):
    with open(path, 'w', encoding = 'UTF-8') as f:
        for line in lines:
            f.write(f'{line}\n')

write_lines("content/train.txt", nsmc.train.get_all_texts())
write_lines("content/test.txt", nsmc.test.get_all_texts())

### GPT 토크나이저 구축

In [4]:
import os

os.makedirs("bbpe", exist_ok=True)

In [6]:
from tokenizers import ByteLevelBPETokenizer

bytebpe_tokenizer = ByteLevelBPETokenizer()

bytebpe_tokenizer.train(
    files = ["content/train.txt", "content/test.txt"], # 학습 말뭉치 리스트 형태로 삽입
    vocab_size = 10000, # 어휘 집합 크기 조절
    special_tokens = ["[PAD]"] # 특수 토큰 추가
)

bytebpe_tokenizer.save_model("bbpe")

['bbpe\\vocab.json', 'bbpe\\merges.txt']

### Bert 토크나이저 구축

In [7]:
import os

os.makedirs("wordpiece", exist_ok=True)

In [8]:
from tokenizers import BertWordPieceTokenizer

wordpiece_tokenizer = BertWordPieceTokenizer(lowercase = False)
wordpiece_tokenizer.train(
    files = ["content/train.txt","content/test.txt"],
    vocab_size = 10000
)

wordpiece_tokenizer.save_model("wordpiece")

['wordpiece\\vocab.txt']

### GPT 토큰화

In [9]:
from transformers import GPT2Tokenizer

tokenizer_gpt = GPT2Tokenizer.from_pretrained("bbpe")
tokenizer_gpt.pad_token = "[PAD]"

file bbpe\config.json not found


In [13]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠.. 포스터보고 초딩영화줄,,, 오버연기조차 가볍지 않구나",
    "심금을 울리는 영화다",
    "둘이서 보다가 하나가 죽어도 모르겠다",
    "이제껏 이런 성향의 영화를 본 적이 없다 "
]

tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]
print(tokenized_sentences)

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'], ['íĿł', '..', 'Ġíı¬ìĬ¤íĦ°', 'ë³´ê³ł', 'Ġì´ĪëĶ©', 'ìĺģíĻĶ', 'ì¤Ħ', ',,,', 'Ġìĺ¤ë²Ħ', 'ìĹ°ê¸°', 'ì¡°ì°¨', 'Ġê°Ģë³į', 'ì§Ģ', 'ĠìķĬ', 'êµ¬ëĤĺ'], ['ìĭ¬', 'ê¸Ī', 'ìĿĦ', 'Ġìļ¸ë¦¬ëĬĶ', 'ĠìĺģíĻĶëĭ¤'], ['ëĳĺ', 'ìĿ´', 'ìĦľ', 'Ġë³´ëĭ¤ê°Ģ', 'ĠíķĺëĤĺê°Ģ', 'Ġì£½', 'ìĸ´ëıĦ', 'Ġëª¨ë¥´ê²łëĭ¤'], ['ìĿ´ìłľ', 'ê»ı', 'ĠìĿ´ëŁ°', 'ĠìĦ±', 'íĸ¥', 'ìĿĺ', 'ĠìĺģíĻĶë¥¼', 'Ġë³¸', 'Ġìłģ', 'ìĿ´', 'ĠìĹĨëĭ¤', 'Ġ']]


In [11]:
batch_input = tokenizer_gpt(
    sentences,
    padding = "max_length", # 문장의 최대 길이에 맞춰 패딩
    max_length = 12,        # 문장의 토큰 기준 최대 길이
    truncation = True       # 문장 잘림 허용 여부
)

# attention_mask : 일반 토큰이 자리한 곳과 패딩 토큰이 자리한 곳을 구분해 알려주는 장치

In [14]:
batch_input["input_ids"]

[[335, 2339, 264, 582, 4056, 465, 3809, 0, 0, 0, 0, 0],
 [3694, 264, 2721, 759, 2884, 357, 807, 1908, 4445, 876, 2961, 7293],
 [670, 617, 314, 6993, 1201, 0, 0, 0, 0, 0, 0, 0],
 [3519, 265, 319, 1877, 9348, 917, 1554, 2133, 0, 0, 0, 0],
 [2311, 3290, 650, 964, 1589, 307, 778, 627, 1285, 265, 898, 221]]

In [15]:
batch_input["attention_mask"]

[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

### Bert 토큰화

In [19]:
from transformers import BertTokenizer

tokenizer_bert = BertTokenizer.from_pretrained(
    "wordpiece", do_lower_case = False
)

file wordpiece\config.json not found


In [20]:
tokenized_sentences_bert = [tokenizer_bert.tokenize(sentence) for sentence in sentences]

batch_inputs = tokenizer_bert(
    sentences,
    padding = "max_length",
    max_length = 12,
    truncation = True
)

In [21]:
batch_inputs["input_ids"]

[[2, 621, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0],
 [2, 997, 16, 16, 2609, 2045, 2796, 1981, 1049, 14, 14, 3],
 [2, 601, 1036, 1155, 6440, 2195, 3, 0, 0, 0, 0, 0],
 [2, 5014, 1074, 2382, 8358, 8422, 1083, 2693, 3, 0, 0, 0],
 [2, 2233, 1464, 1999, 564, 1251, 1063, 2038, 503, 9035, 2080, 3]]

- CLS : 모든 문장 앞에 대응하는 인덱스(2)
- SEP : 모든 문장 뒤에 대응하는 인덱스(3)
- Bert는 문장 시작과 끝에 이 2개 토큰을 덧붙임

In [23]:
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [22]:
batch_inputs["token_type_ids"]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]